# P2 MAAAI 

PREPROCESADO DE DATOS:

In [18]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [24]:

# Cargar el conjunto de datos CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

print(f"Entrenamiento: {x_train.shape}, Prueba: {x_test.shape}")

# Eliminamos el 80% de las etiquetas en el conjunto de entrenamiento
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

# Creamos las particiones de datos
x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]
y_train_unlabeled = y_train[unlabeled_indices]  

print(f"Instancias de entrenamiento etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")


Entrenamiento: (50000, 32, 32, 3), Prueba: (10000, 32, 32, 3)
Instancias de entrenamiento etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


CREACIÓN DE LA RED NEURONAL CONVOLUCIONAL

In [25]:
from tensorflow.keras import layers, models

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

def crear_modelo():
    model = models.Sequential()
    
    # Capa convolucional 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.0001), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Aplanamos las salidas
    model.add(layers.Flatten())
    
    # Capa densa 1
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    
    # Capa densa 2
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    
    # Capa de salida
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compilamos el modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = crear_modelo()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [37]:
#ENTRENAMIENTO
history = model.fit(x_train_labeled, y_train_labeled, epochs=10, batch_size=256)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 874ms/step - accuracy: 0.0230 - loss: 5.6206
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.0553 - loss: 4.8857
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 35s 877ms/step - accuracy: 0.0876 - loss: 4.5168
Epoch 4/10
12/40 ━━━━━━━━━━━━━━━━━━━━ 25s 893ms/step - accuracy: 0.1006 - loss: 4.2867

KeyboardInterrupt: 

In [ ]:
#CONJUNTO DE PRUEBA
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print(f"Rendimiento en la prueba: Pérdida = {test_loss}, Precisión = {test_acc}")

313/313 - 3s - 9ms/step - accuracy: 0.1421 - loss: 4.0810
Rendimiento en la prueba: Pérdida = 4.081023693084717, Precisión = 0.1421000063419342
